In [311]:
# from numba import jit

In [312]:
import re
import nltk
from imdb import IMDb
from difflib import SequenceMatcher

OFFICIAL_AWARDS_1315 = ['cecil b. demille award', 'best motion picture - drama', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best motion picture - comedy or musical', 'best performance by an actress in a motion picture - comedy or musical', 'best performance by an actor in a motion picture - comedy or musical', 'best animated feature film', 'best foreign language film', 'best performance by an actress in a supporting role in a motion picture', 'best performance by an actor in a supporting role in a motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best television series - comedy or musical', 'best performance by an actress in a television series - comedy or musical', 'best performance by an actor in a television series - comedy or musical', 'best mini-series or motion picture made for television', 'best performance by an actress in a mini-series or motion picture made for television', 'best performance by an actor in a mini-series or motion picture made for television', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']
OFFICIAL_AWARDS_1819 = ['best motion picture - drama', 'best motion picture - musical or comedy', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best performance by an actress in a motion picture - musical or comedy', 'best performance by an actor in a motion picture - musical or comedy', 'best performance by an actress in a supporting role in any motion picture', 'best performance by an actor in a supporting role in any motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best motion picture - animated', 'best motion picture - foreign language', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best television series - musical or comedy', 'best television limited series or motion picture made for television', 'best performance by an actress in a limited series or a motion picture made for television', 'best performance by an actor in a limited series or a motion picture made for television', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best performance by an actress in a television series - musical or comedy', 'best performance by an actor in a television series - musical or comedy', 'best performance by an actress in a supporting role in a series, limited series or motion picture made for television', 'best performance by an actor in a supporting role in a series, limited series or motion picture made for television', 'cecil b. demille award']

In [313]:
list(set(OFFICIAL_AWARDS_1315) - set(OFFICIAL_AWARDS_1819))

['best performance by an actor in a motion picture - comedy or musical',
 'best performance by an actress in a television series - comedy or musical',
 'best animated feature film',
 'best performance by an actor in a television series - comedy or musical',
 'best television series - comedy or musical',
 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television',
 'best performance by an actress in a supporting role in a motion picture',
 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television',
 'best mini-series or motion picture made for television',
 'best motion picture - comedy or musical',
 'best performance by an actress in a mini-series or motion picture made for television',
 'best foreign language film',
 'best performance by an actor in a supporting role in a motion picture',
 'best performance by an actress in a motion picture - comedy or musical',
 'best per

In [314]:
import itertools
import json

def get_tweets(year):
    try:
        if year == "2013" or year == "2015" or year == "2018" or year == "2019":
            f = open('gg'+year+'.json')
            data = json.load(f)
            TWEETS = [tweet['text'] for tweet in data]
            TWEETS.sort()
            TWEETS = list(TWEETS for TWEETS,_ in itertools.groupby(TWEETS))
            return TWEETS
        else:                                              ## need to know 1819 open methods???????????
            with open('gg'+year+'.json', encoding='utf8') as json_file:
                data = [json.loads(line) for line in json_file]
            TWEETS = [tweet['text'] for tweet in data]

            TWEETS.sort()
            TWEETS = list(TWEETS for TWEETS,_ in itertools.groupby(TWEETS))
            return TWEETS
    except:
        return False

In [315]:

def divide_tweets(year):

    print("\nProcessing data of year {}".format(year))
    
    tweets = get_tweets(year)
    tweet_by_award_dict = dict()
    
    if year == "2013" or year == "2015": #or year == "2020":
        OFFICIAL_AWARDS = OFFICIAL_AWARDS_1315
    else: 
        OFFICIAL_AWARDS = OFFICIAL_AWARDS_1819
        
    for award in OFFICIAL_AWARDS:
        tweet_by_award_dict[award] = []

    to_delete = ['-', 'a', 'an', 'award', 'best', 'by', 'for', 'in', 'made', 'or', 'performance', 'role',
                 'feature', 'language']

    fresh_names = dict()
    for award in OFFICIAL_AWARDS:
        fresh_names[award] = [[item for item in award.split() if not item in to_delete]]

    for award in OFFICIAL_AWARDS:
        if "television" in award:
            for word in ['tv', 't.v.']:
                extra = award.replace("television", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])

        if "motion picture" in award:
            for word in ['movie', 'film']:
                extra = award.replace("motion picture", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])


        if "film" in award:
            for word in ['motion picture', 'movie']:
                extra = award.replace("film", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])

        if "comedy or musical" in award:
            for word in ['comedy', 'musical']:
                extra = award.replace("comedy or musical", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])

    

        if "series, mini-series or motion picture made for television" in award:
            for word in ['series', 'mini-series', 'miniseries', 'tv', 'television', 'tv movie', 'tv series', 'television series']:
                extra = award.replace("series, mini-series or motion picture made for television", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])

            

        if "mini-series or motion picture made for television" in award:
            for word in ['television movie', 'mini-series', 'miniseries', 'tv movie']:
                extra = award.replace("mini-series or motion picture made for television", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])

        if "television series" in award:
            for word in ['tv', 't.v.', 'television', 'series']:
                extra = award.replace("television series", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])


        if "television series - comedy or musical" in award:

            for word in ["tv comedy", "tv musical", "comedy series", "t.v. comedy", "t.v. musical", "television comedy", "television musical"]:
                extra = award.replace("television series - comedy or musical", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])

        if "television series - drama" in award:
            for word in ["tv drama", "drama series", "television drama", "t.v. drama"]:
                extra = award.replace("television series - drama", word)
                fresh_names[award].append([item for item in extra.split() if not item in to_delete])

    OFFICIAL_AWARDS.sort(key=lambda s: len(s), reverse=True)

    for award in OFFICIAL_AWARDS:
        tweet_length = len(tweets)
        for i in range(tweet_length - 1, -1, -1):
            tweet = tweets[i]
            for extra in fresh_names[award]:
                flag = True
                for word in extra:
                    if flag == True:
                        flag = flag and word.lower() in tweet.lower()

                if flag == True:
                    tweet_by_award_dict[award].append(tweet)
                    del tweets[i]
                    break


    return tweet_by_award_dict

In [316]:

def get_presenters(year):
    #code for presenters
    ia = IMDb()

    award_tweet_dict = divide_tweets(year)
    print ("Done Preprocessing...")
    
    presenters_dict_by_awards = {}
    stop = ["original", "originals", "best", "screenplay", "feature", "animated", "actress", "motion", "picture", 'lifetime', "series", "comedy", "actor", "supporting", "movie", "foreign", "golden", "award", "goldenglobes", "globes", "film"]

    single_presenter_pattern = re.compile(r'[A-Z][a-z]+\s[A-Z][a-z]+(?=\spresent|\sis\spresenting|\sis\sintroducing|\sgive|\sis\sgiving|\spronounce|\sis\spronouncing|\saward|\sis\sawarding)')
    multiple_presenters_pattern = re.compile(r'[A-Z][a-z]+\s[A-Z][a-z]+\sand\s[A-Z][a-z]*.*\s*[A-Z][a-z]+(?=\spresent|\sintroduc|\sare\sintroducing|\sgive|\sare\sgiving|\spronounce|\sare\spronouncing|\saward|\sare\sawarding)')
        
    for award in award_tweet_dict:
        presenters_dict_by_awards[award] = []

        for tweet in award_tweet_dict[award]:
            tweet = re.sub(r'&amp;', 'and', tweet)
            tweet = re.sub(r'\s+@\w+', '', tweet)
            multiple_presenters = re.findall(multiple_presenters_pattern, tweet)

            for presenter in multiple_presenters:
                pp = presenter.split(' and ')
                p1 = pp[0]
                if any(word in p1.lower() for word in stop):
                    continue

                pp = presenter.split(' and ')
                pt = pp[1]
                ptt = pt.split(' ')
                pttname = ptt[0:2]
                p2 = ' '.join(pttname)
                if any(word in p2.lower() for word in stop):
                    continue

                person = ia.search_person(p1)
                if person:
                    p1 = person[0]['name'].lower()
                person = ia.search_person(p2)
                if person:
                    p2 = person[0]['name'].lower()
                if p1 not in presenters_dict_by_awards[award]:
                    presenters_dict_by_awards[award].append(p1)
                if p2 not in presenters_dict_by_awards[award]:
                    presenters_dict_by_awards[award].append(p2)

            single_presenter = re.findall(single_presenter_pattern, tweet)
            for presenter in single_presenter:
                if any(word in presenter.lower() for word in stop):
                    continue
                person = ia.search_person(presenter)
                if person:
                    presenter = person[0]['name'].lower()
                if presenter not in presenters_dict_by_awards[award]:
                    presenters_dict_by_awards[award].append(presenter)

    return presenters_dict_by_awards

In [317]:
year = "2015"
presenter_list = get_presenters(year)
print (presenter_list)


Processing data of year 2015
Done Preprocessing...
{'cecil b. demille award': ['julianna margulies', 'robert downey jr.', 'don cheadle', 'robert de niro', 'al pacino', 'george clooney', 'caryl hathaway', 'Alicia Florrick', 'earl rhodes', 'marty krofft'], 'best motion picture - drama': ['meryl streep', 'jack black', 'clive owen', 'channing tatum'], 'best performance by an actress in a motion picture - drama': ['julianne moore', 'matthew mcconaughey'], 'best performance by an actor in a motion picture - drama': ['gwyneth paltrow'], 'best motion picture - comedy or musical': ['robert downey jr.'], 'best performance by an actress in a motion picture - comedy or musical': ['ricky gervais'], 'best performance by an actor in a motion picture - comedy or musical': ['amy adams'], 'best animated feature film': ['salma hayek', 'kevin hart', 'kevin heart'], 'best foreign language film': ['colin farrell'], 'best performance by an actress in a supporting role in a motion picture': ['jared leto'], '

In [131]:
len(presenter_list)

26

In [150]:
award_tweet_dict = divide_tweets(year)


Processing data of year 2020


In [297]:
for award in award_tweet_dict:
    for tweet in award_tweet_dict[award]:
        if "sandra bullock" in tweet.lower():# and "actor" in tweet.lower():
            print (tweet)
            
            #tweet = re.sub(r'&amp;', 'and', tweet)
            #tweet = re.sub(r'\s+@\w+', '', tweet)
            #print (tweet)

Golden Girl. Sandra Bullock lights up the night presenting Best Motion Picture, Drama at the 77th Annual #GoldenGlobes in a custom ochre silk moiré faille gown. Photographed by @bryrandall. https://t.co/AzKfEzIz09
Fashion And Fitness Top story: @OscardelaRenta: 'Golden Girl. Sandra Bullock lights up the night presenting Best Motion Picture, Drama at the 77th Annual #GoldenGlobes in a custom ochre silk moiré faille gown. Photograp… https://t.co/eWDiRNwuXw, see more https://t.co/2bEF4kyiUd


In [296]:
import goslate
tweets = get_tweets(year)
#from translate import Translator
#translator = Translator(from_lang='autodetect', to_lang='en')


for tweet in tweets:
    #tweet = translator.translate(tweet)
    if "sandra bullock" in tweet.lower() and "drama" in tweet.lower():
        print (tweet)

Fashion And Fitness Top story: @OscardelaRenta: 'Golden Girl. Sandra Bullock lights up the night presenting Best Motion Picture, Drama at the 77th Annual #GoldenGlobes in a custom ochre silk moiré faille gown. Photograp… https://t.co/eWDiRNwuXw, see more https://t.co/2bEF4kyiUd
Golden Girl. Sandra Bullock lights up the night presenting Best Motion Picture, Drama at the 77th Annual #GoldenGlobes in a custom ochre silk moiré faille gown. Photographed by @bryrandall. https://t.co/AzKfEzIz09
When you didn't realize parasite wasn't nominated for best drama until you are watching Sandra Bullock read the nominees during the golden globes
https://t.co/EnCPlM2QtO  Talk about a dramatic win! Sandra Bullock presented the last award of the night at Sunday's 2020 Golden Globe Aw... https://t.co/0PZtCN5zsI
https://t.co/yL6A4a8OYl Talk about a dramatic win!
Sandra Bullock presented the last award of the night at Sunday's 2020 Golden Globe Aw... https://t.co/bByAWZm4Qz


In [12]:
len(winner_list)

26

In [ ]:
personaward = [] # 16
for award in OFFICIAL_AWARDS_1315:
        for person in ["actor", "actress", "demille", "director"]:
            if person in award.split():
                personaward.append(award)
                

In [ ]:
thingaward = []
for award in OFFICIAL_AWARDS_1315:
        if award not in personaward:
            thingaward.append(award)

In [ ]:
thingaward

In [34]:
from imdb import IMDb

# create an instance of the IMDb class
ia = IMDb()

# get a movie and print its director(s)
the_matrix = ia.get_movie('0133093')
for director in the_matrix['directors']:
    print(director['name'])

# show all information that are currently available for a movie
#print(sorted(the_matrix.keys()))



Lana Wachowski
Lilly Wachowski


In [38]:
movies = ia.search_movie('amour austria')

In [39]:
movies

[<Movie id:1076835[http] title:_A Mountain Torrent in Austria (1911)_>,
 <Movie id:5102928[http] title:_"Second City Headlines & News" Writing Czechs Your Austria Can't Cash (1997)_>,
 <Movie id:1876032[http] title:_"The Amazing Race" I Cannot Deal with Your Psycho Behavior: Austria (2011)_>,
 <Movie id:4697402[http] title:_"Die Promicamper" Prince Mario-Max Schaumburg-Lippe, Sepp Resnik, Yvonne Rueff and Davorka Tovilo tour Austria (2011)_>,
 <Movie id:4697408[http] title:_"Die Promicamper" Prince Mario-Max Schaumburg-Lippe, Sepp Resnik, Yvonne Rueff and Davorka Tovilo tour Austria (2011)_>,
 <Movie id:4697398[http] title:_"Die Promicamper" Prince Mario-Max Schaumburg-Lippe, Sepp Resnik, Yvonne Rueff and Davorka Tovilo tour Austria (2015)_>,
 <Movie id:4697392[http] title:_"Die Promicamper" Prince Mario-Max Schaumburg-Lippe, Sepp Resnik, Yvonne Rueff and Davorka Tovilo tour Austria (2011)_>,
 <Movie id:4697412[http] title:_"Die Promicamper" Prince Mario-Max Schaumburg-Lippe, Sepp Resn

In [40]:
for item in movies:
    if item['year'] == 2012:
        print (item['title'])
    print (item['year'])

1911
1997
2011
2011
2011
2015
2011
2011
2011
2011
2011
2011
2011


In [47]:
act_names = ia.search_person('stellan skarsg')

In [50]:
act_names[0]['name']

'Stellan Skarsgård'

In [51]:
 print (SequenceMatcher(None, 'stellan skarsg',act_names[0]['name'].lower()).ratio())

0.9032258064516129


In [52]:
 print (SequenceMatcher(None, 'stellan skarsg',act_names[1]['name'].lower()).ratio())

0.7857142857142857
